In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from matplotlib import pyplot as plt
import os
import time
import pandas as pd
import subprocess
import os
from datetime import datetime,timedelta
from datetime import date
import urllib.request
from PIL import Image

In [ ]:
print('----------------------- Libraries are imported ---------------------------')
 
### Import Data and Create Train and test sets
 
torch.manual_seed(42)
torch.cuda.manual_seed(42)
 
## Finds images
train_data_path = '../input/gan-getting-started/'
 
### Rescaling incoming image to 28 by 28 pixels
### After Rescaling, convert the image to a tensor
transform = transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=transform)
test_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=transform)
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data,batch_size,shuffle=True)
test_loader = train_loader
##### Declare the model architecture
 
d = 20

In [ ]:
d = 20
 
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        
        
        
        self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
        nn.BatchNorm2d(3),
        nn.Conv2d(in_channels=3,out_channels=2,kernel_size=3,padding=1,stride=1),
        nn.ReLU(),
        nn.BatchNorm2d(2)
    
            
        )
        self.resnet_adder = nn.Sequential(
            nn.ReLU(),
        nn.BatchNorm2d(3)
        )
        
        self.fc1 = nn.Sequential(
            ### Reduce the number of channels to 1 without changing the width and dimensions of the images
            nn.Linear(128*128*2,128),
            
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128,64),
            
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,32)
            
        )
 
        self.encoder = nn.Sequential(
            
            nn.Linear(32, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, d * 2)
        )
        
 
        self.decoder = nn.Sequential(
            nn.Linear(d, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, 32)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(32,64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64,128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128,128*128*2),
            nn.ReLU(),
            nn.BatchNorm1d(128*128*2)
            
        )
        self.tconv1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=2,out_channels=3,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(3),
            nn.ConvTranspose2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1),
            nn.Sigmoid()
        
        )
 
    def reparameterise(self, mu, logvar):
        if self.training:
            ## Using log variance to ensure that we get a positive std dev
            ## Converting to std dev in the real space
            std = logvar.mul(0.5).exp_()
            ### Create error term which has the same shape as std dev sampled from a N(0,1) distribution
            eps = std.data.new(std.size()).normal_()
            #eps = torch.zeros(std.size())
            ### Add the mean and the std_dev 
            return eps.mul(std).add_(mu)
        else:
            return mu
 
    def forward(self, x):
        
        conv1_output = self.conv1(x)
        #conv1_output = self.resnet_adder(conv1_output + x) 
        fc1_output = self.fc1(conv1_output.view(-1,128*128*2))
        
        ### Convert Encoded vector into shape (N,2,d)
        mu_logvar = self.encoder(fc1_output).view(-1, 2, d)
        ### First vector for each image is mean of the latent distribution
        mu = mu_logvar[:, 0, :]
        ### Second vector for each image is log-variance of the latent distribution
        logvar = mu_logvar[:, 1, :]
        ### Create variable Z = mu + error * Std_dev
        z = self.reparameterise(mu, logvar)
        ### Get decoder output
        decoder_output = self.decoder(z)
        
        fc2_output = self.fc2(decoder_output)
        tconv1_output = self.tconv1(fc2_output.view(fc2_output.size(0),2,128,128))
        ## Resize Decoder Output to Pass it to TransposedConv2d layer to recontruct 3 channeled image
        #decoder_output = decoder_output.view(decoder_output.size(0),1,28,28) 
        ## Return Reconstructed Output and mean and log-variance
        return tconv1_output, mu, logvar,z
    
model = VAE()

In [ ]:
learning_rate = 3e-3
 
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate
    )
### Loss function
 
#Reconstruction + KL divergence losses summed over all elements and batch
 
def loss_function(x_hat, x, mu, logvar):
    MSE = nn.MSELoss(reduction='sum')
    ## Making sure that distributions do not overlap
#     loss = nn.functional.binary_cross_entropy(
#         x_hat, x, reduction='sum'
#     )
    loss = 1.0*MSE(x_hat,x)
    #loss = MSE(x_hat,x.view(x.size(0), -1))
#     BCE = nn.functional.binary_cross_entropy(
#         x_hat, x.view(-1, 28*28*3), reduction='sum'
#     )
    ### Makes sure that distributions of each image span entire latent space and the range does not explode
    KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))
 
    return loss + KLD
 
##### Training the model
print('----------------------- Training is started ---------------------------')
 
epochs = 50
codes = dict(μ=list(), logσ2=list(), y=list())
 
loss = {
    'train_loss':[],
    'test_loss' : []
}
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        for x, _ in train_loader:
            #x = x.to(device)
            # ===================forward=====================
            x_hat, mu, logvar,_ = model(x)
            loss = loss_function(x_hat, x, mu, logvar)
            train_loss += loss.item()
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        
        #loss['train_loss'].append(train_loss)
        if epoch % 1 ==0:
            print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
    
    # Testing
    
    means, logvars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x, _ in test_loader:
            #x = x.to(device)
            # ===================forward=====================
            x_hat, mu, logvar,_ = model(x)
            test_loss += loss_function(x_hat, x, mu, logvar).item()
            # =====================log=======================
            means.append(mu.detach())
            logvars.append(logvar.detach())
            #labels.append(y.detach())
    # ===================log========================
    #loss['test_loss'].append(test_loss)
    codes['μ'].append(torch.cat(means))
    codes['logσ2'].append(torch.cat(logvars))
    test_loss /= len(test_loader.dataset)
    if epoch % 1 == 0:
        print(f'====> Test set loss: {test_loss:.4f}')
        #display_images(x, x_hat, 1, f'Epoch {epoch}')
        #plt.show()
 
#### Extract embeddings
print('----------------------- Training has ended ---------------------------')
 
 

In [ ]:
### Generating Images From Samples
import numpy as np
from torch.nn.modules.upsampling import Upsample
import matplotlib.pyplot as plt
for i in range(10):
    print(i)
    N = 1
    z = torch.randn((N,d))
    decoder_output = model.decoder(z)        
    fc2_output = model.fc2(decoder_output)
    tconv1_output = model.tconv1(fc2_output.view(N,2,128,128))
    trans = transforms.ToPILImage()
    trans1 = Upsample(scale_factor = 2, mode='nearest')
    tconv1_output = trans1(tconv1_output)
    tconv1_output = tconv1_output.view(3,256,256)
    img = trans(tconv1_output)
    plt.imshow(np.asarray(img))
    plt.pause(0.5)
    

In [ ]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}
 
torch.save(checkpoint, 'checkpoint.pth')
 
print('----------------------- Save the Model ---------------------------')